In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from models.detector import face_detector
from models.detector.iris_detector import IrisDetector
import sys
import warnings
warnings.filterwarnings("ignore")

C:\Users\jason\anaconda3\envs\keras224\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jason\anaconda3\envs\keras224\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jason\anaconda3\envs\keras224\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jason\anaconda3\envs\keras224\lib\site-packages\tensorf

In [2]:
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-1_keras.h5",
     fd_type="s3fd"
    # 2DFAN-4_keras.h5, 2DFAN-1_keras.h5
)

Instructions for updating:
Colocations handled automatically by placer.
loading S3FD model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, None, None, 6 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 6 0           conv1_2[0][0]                    
______

In [3]:
def resize_image(im, max_size=768):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
    return im

In [4]:
# Test images are obtained on https://www.pexels.com/
im = cv2.imread(r"C:\Users\jason\Facial-Recognition-FaceNet-master\database\id\eric.jpg")[..., ::-1]
im = resize_image(im) # Resize image to prevent GPU OOM.
bboxes, landmarks = fd.detect_face(im, with_landmarks=True)

Resize image to (768, 432).
Building FAN for landmarks detection...
loading FAN model............
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
conv10.23005182903749222_pad (Z (None, 3, 262, 262)  0           input0[0][0]                     
__________________________________________________________________________________________________
conv10.23005182903749222 (Conv2 (None, 64, 128, 128) 9472        conv10.23005182903749222_pad[0][0
__________________________________________________________________________________________________
bn10.20053864458101422 (BatchNo (None, 64, 128, 128) 256         conv10.23005182903749222[0][0]   
___________

In [5]:
import math
def cal_two_points_distance(p1, p2):
    return math.sqrt(math.pow((p2[0] - p1[0]), 2) + math.pow((p2[1] - p1[1]), 2))

In [6]:
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    sys.exit("Could not open webcam, please open the camera!")
count = 0
while True:
    count= count+1
    # read frame from webcam 
    status, frame = webcam.read()     
    frame_size = frame.shape
    face, landmarks = fd.detect_face(frame, with_landmarks=True)
    number = len(face)
    
    if number!=0:
        
        print(f"detect {number} face(s)!")
        print(face)
        print(" ")
        print("landmarks:",landmarks)
        
       
        #計算距離
        x1,y1 = landmarks[0][2]
        x2,y2 = landmarks[0][30]
        x3,y3 = landmarks[0][14]

        left_dist = round(cal_two_points_distance((y1,x1),(y2,x2)), 2)
        right_dist = round(cal_two_points_distance((y2,x2),(y3,x3)), 2)        
 
        ratio = left_dist/right_dist


        if ratio>2:
            status = "left"
        elif ratio<0.5:
            status = "right"
        else:
            status = "front"
 
        if y2>=y3 or y2<=y1:
            cv2.putText(frame, f"face angle too large!!", (10, 90), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
   
        #畫出點
        for i in range(number):
            
            #處理臉部 bbox
            x0, y0, x1, y1, score = face[i] 
            x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
            print(x0,y0,x1,y1)
            cv2.rectangle(frame, (y0, x0), (y1, x1), (0, 255, 0), 2)
                
            for j in range(len(landmarks[i])): 
                

                #處理臉部 landmarks
                x, y = landmarks[i][j]
                
                if j==14 or j==2:
                    
                    cv2.circle(frame, (int(y), int(x)), 3, (0,0,255), -1)
                elif j==30:
                    
                    cv2.circle(frame, (int(y), int(x)), 3, (255,0,0), -1)
                    
                else:
                    cv2.circle(frame, (int(y), int(x)), 3, (0,255,0), -1)
                    
            cv2.putText(frame, f"left_dist:{left_dist}, right_dist:{right_dist}", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            cv2.putText(frame, f"status:{status}", (10, 50), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            cv2.putText(frame, f"ratio:{ratio}", (10, 70), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            
    else:
        
        print("do not detect face(s))!")

    cv2.namedWindow("demo", cv2.WINDOW_NORMAL)
    cv2.imshow("demo", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()    

detect 1 face(s)!
[array([217.96251678466797, 274.95901679992676, 435.5796890258789,
       449.7168140411377, array([0.9996863], dtype=float32)], dtype=object)]
 
landmarks: [array([[320., 271.],
       [342., 278.],
       [364., 290.],
       [380., 296.],
       [402., 309.],
       [411., 331.],
       [411., 353.],
       [411., 375.],
       [408., 391.],
       [395., 432.],
       [402., 432.],
       [389., 444.],
       [376., 451.],
       [354., 451.],
       [332., 448.],
       [314., 448.],
       [288., 444.],
       [285., 296.],
       [273., 309.],
       [266., 322.],
       [266., 334.],
       [269., 344.],
       [260., 394.],
       [257., 407.],
       [254., 416.],
       [257., 429.],
       [263., 432.],
       [301., 375.],
       [317., 375.],
       [323., 375.],
       [339., 385.],
       [342., 375.],
       [345., 378.],
       [348., 385.],
       [348., 385.],
       [345., 385.],
       [307., 322.],
       [301., 328.],
       [301., 337.],
     

detect 1 face(s)!
[array([237.7639856338501, 289.59765815734863, 448.66936588287354,
       450.21943855285645, array([0.99862194], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[333., 284.],
       [357., 293.],
       [375., 299.],
       [393., 302.],
       [414., 308.],
       [422., 326.],
       [422., 344.],
       [417., 359.],
       [417., 370.],
       [411., 382.],
       [408., 406.],
       [405., 433.],
       [387., 445.],
       [366., 451.],
       [345., 451.],
       [327., 454.],
       [306., 451.],
       [297., 296.],
       [291., 308.],
       [288., 317.],
       [288., 326.],
       [288., 335.],
       [279., 385.],
       [276., 403.],
       [273., 412.],
       [265., 448.],
       [276., 439.],
       [303., 356.],
       [321., 359.],
       [336., 365.],
       [339., 365.],
       [351., 359.],
       [339., 359.],
       [351., 365.],
       [348., 370.],
       [345., 373.],
       [318., 314.],
       [309., 320.],
       [309., 329.

detect 1 face(s)!
[array([239.80650329589844, 290.9221541881561, 441.8231201171875,
       441.3643386363983, array([0.9995943], dtype=float32)], dtype=object)]
 
landmarks: [array([[328., 284.],
       [353., 287.],
       [373., 296.],
       [390., 299.],
       [410., 304.],
       [421., 321.],
       [421., 341.],
       [419., 352.],
       [419., 369.],
       [416., 381.],
       [413., 400.],
       [404., 426.],
       [390., 440.],
       [370., 446.],
       [348., 446.],
       [331., 448.],
       [311., 446.],
       [297., 293.],
       [294., 307.],
       [291., 313.],
       [288., 321.],
       [288., 332.],
       [269., 440.],
       [266., 443.],
       [266., 443.],
       [269., 446.],
       [277., 443.],
       [308., 349.],
       [317., 352.],
       [325., 352.],
       [336., 361.],
       [328., 316.],
       [331., 318.],
       [339., 341.],
       [342., 341.],
       [345., 341.],
       [314., 310.],
       [308., 316.],
       [305., 324.],
      

detect 1 face(s)!
[array([228.83318424224854, 286.10124015808105, 458.5730981826782,
       455.27197074890137, array([0.9999757], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[310., 285.],
       [345., 282.],
       [364., 285.],
       [384., 288.],
       [403., 292.],
       [422., 301.],
       [425., 304.],
       [425., 311.],
       [435., 349.],
       [432., 372.],
       [435., 397.],
       [438., 420.],
       [416., 439.],
       [390., 448.],
       [364., 455.],
       [345., 458.],
       [320., 461.],
       [297., 304.],
       [294., 314.],
       [291., 324.],
       [291., 330.],
       [297., 340.],
       [294., 388.],
       [294., 404.],
       [294., 413.],
       [297., 426.],
       [307., 439.],
       [326., 362.],
       [342., 362.],
       [361., 359.],
       [368., 365.],
       [393., 362.],
       [384., 362.],
       [387., 365.],
       [377., 375.],
       [377., 375.],
       [323., 317.],
       [323., 324.],
       [320., 333.]

detect 1 face(s)!
[array([219.88978576660156, 225.49816131591797, 467.9965057373047,
       412.60469818115234, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[339., 222.],
       [363., 229.],
       [384., 243.],
       [402., 254.],
       [426., 267.],
       [447., 288.],
       [454., 309.],
       [464., 330.],
       [468., 362.],
       [450., 389.],
       [430., 400.],
       [412., 410.],
       [391., 417.],
       [363., 417.],
       [339., 417.],
       [308., 414.],
       [283., 407.],
       [314., 233.],
       [304., 243.],
       [297., 254.],
       [294., 264.],
       [294., 278.],
       [276., 337.],
       [269., 348.],
       [262., 358.],
       [262., 369.],
       [266., 382.],
       [318., 320.],
       [342., 323.],
       [356., 327.],
       [370., 330.],
       [388., 320.],
       [388., 327.],
       [384., 334.],
       [381., 344.],
       [377., 351.],
       [335., 260.],
       [332., 267.],
       [328., 278.],
       [332

...

detect 1 face(s)!
[array([209.4019260406494, 233.22519493103027, 452.1412296295166,
       431.5660457611084, array([0.99999905], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[327., 238.],
       [359., 241.],
       [377., 248.],
       [394., 256.],
       [412., 270.],
       [419., 294.],
       [419., 312.],
       [419., 323.],
       [423., 347.],
       [398., 372.],
       [405., 393.],
       [398., 411.],
       [384., 425.],
       [363., 432.],
       [345., 432.],
       [317., 432.],
       [292., 429.],
       [296., 241.],
       [288., 252.],
       [281., 266.],
       [281., 277.],
       [281., 291.],
       [274., 351.],
       [267., 365.],
       [264., 376.],
       [264., 390.],
       [271., 404.],
       [317., 330.],
       [341., 330.],
       [363., 333.],
       [373., 333.],
       [384., 330.],
       [384., 337.],
       [384., 344.],
       [380., 351.],
       [380., 358.],
       [324., 266.],
       [324., 277.],
       [320., 287.],

detect 1 face(s)!
[array([211.6147050857544, 250.85057973861694, 461.7531633377075,
       448.28034353256226, array([0.9999995], dtype=float32)],
      dtype=object)]
 
landmarks: [array([[340., 257.],
       [372., 261.],
       [386., 268.],
       [397., 275.],
       [408., 304.],
       [411., 318.],
       [408., 332.],
       [411., 343.],
       [426., 375.],
       [408., 401.],
       [401., 408.],
       [401., 422.],
       [386., 440.],
       [368., 444.],
       [347., 444.],
       [325., 444.],
       [304., 440.],
       [307., 268.],
       [293., 286.],
       [289., 296.],
       [289., 314.],
       [289., 325.],
       [279., 383.],
       [271., 393.],
       [268., 404.],
       [271., 422.],
       [275., 429.],
       [318., 365.],
       [343., 368.],
       [361., 372.],
       [368., 372.],
       [379., 361.],
       [379., 365.],
       [379., 375.],
       [379., 383.],
       [376., 386.],
       [332., 296.],
       [329., 311.],
       [325., 318.],

detect 1 face(s)!
[array([207.06878662109375, 243.40097904205322, 459.54197692871094,
       443.2427453994751, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[319., 243.],
       [355., 250.],
       [373., 254.],
       [391., 261.],
       [413., 272.],
       [427., 294.],
       [431., 312.],
       [431., 326.],
       [434., 348.],
       [427., 377.],
       [420., 395.],
       [413., 413.],
       [398., 428.],
       [380., 435.],
       [358., 442.],
       [333., 442.],
       [308., 442.],
       [293., 257.],
       [286., 268.],
       [282., 279.],
       [282., 294.],
       [286., 304.],
       [279., 366.],
       [275., 381.],
       [271., 395.],
       [275., 410.],
       [282., 424.],
       [319., 341.],
       [348., 341.],
       [366., 341.],
       [380., 344.],
       [387., 333.],
       [391., 341.],
       [391., 348.],
       [387., 355.],
       [387., 362.],
       [326., 279.],
       [322., 290.],
       [322., 301.],
       [326

detect 1 face(s)!
[array([216.64657425880432, 286.1421756744385, 480.7645857334137,
       484.573450088501, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[299., 279.],
       [336., 283.],
       [358., 283.],
       [377., 290.],
       [399., 305.],
       [425., 331.],
       [440., 346.],
       [455., 368.],
       [466., 394.],
       [458., 427.],
       [455., 446.],
       [436., 457.],
       [414., 471.],
       [392., 479.],
       [369., 483.],
       [344., 486.],
       [318., 486.],
       [295., 309.],
       [288., 323.],
       [288., 338.],
       [292., 353.],
       [299., 368.],
       [292., 420.],
       [288., 434.],
       [284., 449.],
       [284., 460.],
       [292., 475.],
       [336., 397.],
       [362., 397.],
       [381., 397.],
       [395., 397.],
       [403., 386.],
       [403., 390.],
       [407., 401.],
       [403., 409.],
       [399., 416.],
       [332., 331.],
       [332., 346.],
       [329., 353.],
       [332., 

detect 1 face(s)!
[array([218.53907012939453, 269.6364412307739, 469.8292922973633,
       456.5910921096802, array([1.], dtype=float32)], dtype=object)]
 
landmarks: [array([[308., 262.],
       [343., 265.],
       [367., 269.],
       [388., 276.],
       [413., 287.],
       [434., 308.],
       [452., 322.],
       [466., 343.],
       [473., 374.],
       [462., 402.],
       [448., 420.],
       [434., 430.],
       [409., 444.],
       [381., 451.],
       [360., 455.],
       [336., 455.],
       [311., 459.],
       [294., 290.],
       [287., 304.],
       [287., 318.],
       [287., 332.],
       [294., 343.],
       [290., 399.],
       [287., 409.],
       [283., 420.],
       [283., 434.],
       [287., 444.],
       [325., 371.],
       [346., 371.],
       [367., 371.],
       [381., 371.],
       [392., 357.],
       [392., 367.],
       [395., 374.],
       [392., 385.],
       [388., 392.],
       [322., 308.],
       [325., 322.],
       [322., 332.],
       [325.,